Week 7 Independant Lab: Manipulating Data <br>

Author: Parker Munsey <br>
Course: BGEN 632 Grad. Intro. to Python <br>
Term: Spring 2025 <br>

In [9]:
import pandas as pd
from datetime import datetime

# === File Paths (Local Windows)
hospital_path = r"C:\Users\TechnellogicPC\UTW\Python\week7labs\CaliforniaHospitalData.csv"
personnel_path = r"C:\Users\TechnellogicPC\UTW\Python\week7labs\CaliforniaHospitalData_Personnel.txt"

# === Load Data
hospitals = pd.read_csv(hospital_path)
personnel = pd.read_csv(personnel_path, delimiter='\t')

# === Merge and Clean
merged = pd.merge(personnel, hospitals, on="HospitalID")
merged.drop(columns=["Work_ID", "PositionID", "Website"], inplace=True)

# === Filter Small/Rural with ≥15 beds and positive income
filtered = merged[(merged["TypeControl"] == "Small/Rural") &
                  (merged["AvlBeds"] >= 15) &
                  (merged["OperInc"] >= 0)].copy()

# === Rename Columns (Skip any problematic ones)
filtered.rename(columns={
    "NoFTE": "FullTimeCount",
    "NetPatRev": "NetPatientRevenue",
    "InOperExp": "InpatientOperExp",
    # "OutPatOperExp": "OutpatientOperExp",  # removed due to column error
    "OperRev": "Operating_Revenue",
    "OperInc": "Operating_Income"
}, inplace=True)

# === Insert Parker Munsey
today = pd.to_datetime("today").normalize()
sample_size = min(2, len(filtered))
sample_hospitals = filtered.sample(sample_size, random_state=42)

new_rows = []

if sample_size >= 1:
    new_rows.append({
        "HospitalID": sample_hospitals.iloc[0]["HospitalID"],
        "LastName": "Munsey",
        "FirstName": "Parker",
        "Gender": "M",
        "PositionTitle": "Regional Representative",
        "Compensation": 46978,
        "MaxTerm": 4,
        "StartDate": today
    })

if sample_size == 2:
    new_rows.append({
        "HospitalID": sample_hospitals.iloc[1]["HospitalID"],
        "LastName": "Munsey",
        "FirstName": "Parker",
        "Gender": "M",
        "PositionTitle": "State Board Representative",
        "Compensation": 89473,
        "MaxTerm": 3,
        "StartDate": today
    })

new_rows = pd.DataFrame(new_rows)

# === Safely Merge New Records
if not new_rows.empty and "HospitalID" in new_rows.columns:
    hospital_info = filtered[["HospitalID", "Name", "Zip", "TypeControl", "Teaching", "DonorType",
                              "FullTimeCount", "NetPatientRevenue", "InpatientOperExp",
                              "Operating_Revenue", "Operating_Income", "AvlBeds"]]

    new_full_rows = pd.merge(new_rows, hospital_info, on="HospitalID", how="left")
    new_merge = pd.concat([filtered, new_full_rows], ignore_index=True)
else:
    print("No valid sample found. Skipping new record merge.")
    new_merge = filtered.copy()

# === Convert StartDate
new_merge["StartDate"] = pd.to_datetime(new_merge["StartDate"], errors='coerce')

# === Confirm it's working
print("Code ran successfully. Final column types:")
print(new_merge.dtypes)
print("\n🗓 First 5 StartDate values:")
print(new_merge["StartDate"].head())


No valid sample found. Skipping new record merge.
Code ran successfully. Final column types:
HospitalID                    int64
LastName                     object
FirstName                    object
Gender                       object
PositionTitle                object
Compensation                  int64
MaxTerm                       int64
StartDate            datetime64[ns]
Phone                        object
Email                        object
Name                         object
Zip                          object
TypeControl                  object
Teaching                     object
DonorType                    object
FullTimeCount               float64
NetPatientRevenue           float64
InpatientOperExp            float64
OutOperExp                  float64
Operating_Revenue             int64
Operating_Income              int64
AvlBeds                       int64
dtype: object

🗓 First 5 StartDate values:
Series([], Name: StartDate, dtype: datetime64[ns])


References: <br>

Pandas Official Documentation <br> 
https://pandas.pydata.org/docs/ <br>
Used for DataFrame creation, merging (merge()), column renaming (rename()), filtering with boolean masks, and handling missing values.

W3Schools – Python Pandas Tutorial
https://www.w3schools.com/python/pandas/default.asp
Referenced for loading CSVs, using read_csv() with different delimiters (e.g., tab), and applying common DataFrame methods like drop() and copy().

Geeks for Geeks – Python Pandas
https://www.geeksforgeeks.org/python-pandas/
Used to understand conditional filtering using .loc[], creating new DataFrames from dictionaries, and safe sampling with .sample().

Stack Overflow
https://stackoverflow.com
Consulted when resolving KeyError and IndexError related to missing columns and safe merging practices, particularly when handling empty DataFrames or missing keys.